In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install imageio==2.4.1
!pip install imageio-ffmpeg
#!pip install ffmpeg==4.2.7

In [ ]:
!pip install pi_heif

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip 
from moviepy.editor import VideoFileClip

from moviepy.editor import VideoFileClip

In [ ]:
from time import sleep

from google.colab.patches import cv2_imshow

import shutil
import os
import cv2
import moviepy.editor
import re
import csv
import json

import numpy as np

import matplotlib.pyplot as plt

from PIL import Image
import pi_heif

In [ ]:
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import argparse
import imutils
import dlib
from google.colab.patches import cv2_imshow
import math

# CONVERSION OF IMAGES


In [ ]:
source_path = '/content/drive/MyDrive/' #add specific user path
dest_path = '/content/drive/MyDrive/'   #add specific user path

In [ ]:
def converter(SourceFolder, TargetFolder):

  i = 0

  for file in os.listdir(SourceFolder):
    
    SourceFile=SourceFolder + "/" + file
    TargetFile=TargetFolder + "/" + file.replace(".heic",".jpg")

    heif_file = pi_heif.open_heif(SourceFile, convert_hdr_to_8bit=False, bgr_mode=True)
    np_array = np.asarray(heif_file)
    cv2.imwrite(TargetFile, np_array)
    i = i+1
    print(file, i)

In [ ]:
def converter1(SourceFolder, TargetFolder):

  i = 0

  for file in os.listdir(SourceFolder):
    
    SourceFile=SourceFolder + "/" + file
    TargetFile=TargetFolder + "/" + file.replace(".jpg",".png")

    jpg_file = Image.open(SourceFile)
    jpg_file.save(TargetFile)
    i = i+1
    print(TargetFile, i)

In [ ]:
def color2gray(SourceFolder, TargetFolder):
  i = 0
  for file in os.listdir(SourceFolder):
    SourceFile=SourceFolder + "/" + file
    TargetFile=TargetFolder + "/" + file
    img = Image.open(SourceFile).convert('L') #.rotate(90, expand=True)
    img.save(TargetFile)
    i = i+1
    print(file, i)

In [ ]:
#changes path with desidered ones
converter(source_path, dest_path)
converter1(source_path, dest_path)

In [ ]:
color2gray(source_path, dest_path)

# 30-SEC SUBCLIPS CUT


In [ ]:
#to see the video duration
full_video = "/content/drive/MyDrive/"  #add video path
current_duration = VideoFileClip(full_video).duration
print(current_duration)

668.29


In [ ]:
# duration_ms is imposed in way to have a precision division of the video in subclip of 30 sec (there is an approximation) 
# num_chucnk is the final number of subclip if we do subclips of 30 sec
def divide_to_clip(source, dest, duration_ms, num_chunck, flag):

  full_video = source                   #source = path video
  current_duration = duration_ms        #duration in ms of original clip
  print(current_duration)
  divide_into_count = num_chunck        #number of subclip
  single_duration = current_duration/divide_into_count
  print(single_duration)

  current_offset = 0
  i = 0 
  current_video = f"{current_offset}.mp4"

  while current_duration >= current_offset:
    if(flag):                   #flag = 1 external video
      clip = VideoFileClip(full_video).subclip(current_offset, current_offset + single_duration).resize((1920,1080))  
    else:                       #flag = 0 internal video  
      clip = VideoFileClip(full_video).subclip(current_offset, current_offset + single_duration).resize((1080,1920))
    current_offset += single_duration
    i += 1
    current_video = f"{current_offset}.mp4"
    j = str(i)
    clip.write_videofile(dest+j+'_'+current_video, codec="libx264", audio=False)    #dest = path video

    print("-----------------###-----------------")

In [ ]:
divide_to_clip(full_video, "/content/drive/MyDrive/subclips", 660, 22, 1) 

# STATISTICAL TESTS FOR BRIGHTNESS

In [ ]:
def hist_greyscale():

  image_path = '/content/drive/MyDrive/NAPOLI/DATASET_FACE/PNG/crop_resize/'
  gray_values = []
  
  for img in os.listdir(image_path):
    #print(img)
    img_grey = cv2.imread(image_path+img)
    gray_img = cv2.cvtColor(img_grey, cv2.COLOR_BGR2GRAY)
    gray_mean = np.mean(gray_img)
    gray_values.append(gray_mean)
      
  plt.hist(gray_values, bins=50)
  plt.xlabel('Gray Scale Values')
  plt.ylabel('Frequency')
  plt.title('Histogram of Mean Gray Scale Values')
  plt.show()
      

def change():
  
  #to modify brightness and contrast 

  image_path = '/content/drive/MyDrive/NAPOLI/DATASET_FACE/PNG/squared/224/'

  for img in os.listdir(image_path):
      print(image_path+img)
      img_c = cv2.imread(image_path+img)

      # Convert the image to float
      img_float = img_c.astype(float)

      # Increase the contrast of the image
      alpha = 1.1   #contrast increase of 50%
      beta = -64      #brightness decrease of 50%
      img_contrast = cv2.convertScaleAbs(img_float, alpha=alpha, beta=beta) #num*aplha + beta
      print(image_path+img[:-4]+'_change.png')
      cv2.imwrite(image_path+img[:-4]+'_change.png', img_contrast)


# DATASET 

Labels distribution

In [ ]:
import os
import json

label_path = "" #path of MAPILLARY labels

id_labels = {}
list_reg = []

for files in os.listdir(label_path):
  json_f = json.load(open(label_path+files, 'r'))

  for elem in json_f["objects"]:
    label = elem["label"]
    label_s = label.split("--")
    if label_s[0] not in id_labels:
      id_labels[label_s[0]] = 1
    else:
      id_labels[label_s[0]] += 1 
    if "regulatory" in label and label not in list_reg:
      list_reg.append(label)
      print(label)
sum = 0
for elem in id_labels:
    print(elem, id_labels[elem])
    sum += id_labels[elem]

print("dictionary len: ", len(id_labels))

sorted_dict = dict(sorted(id_labels.items(), key=lambda item: item[1]))
for elem in sorted_dict:
  sum += sorted_dict[elem]
  print(elem, sorted_dict[elem])

print("number of elements", sum)

print(len(os.listdir(label_path)))

## CONVERSION FROM JSON LABEL MAPILLARY in YOLO TXT FORMAT

In [ ]:
def convert_value_2_txt(label_path, id_dict_labels):
  k = 0
  path_txt = ''
  for file in os.listdir(label_path+"JSON"):
    
    file_txt = path_txt+file[:-4]+"txt"
    f = open(file_txt, 'w')
    ###
    
    json_f = json.load(open(label_path+"JSON/"+file, 'r'))
    width = json_f["width"]
    height = json_f["height"]
    
    for i in range(len(json_f["objects"])):  
      buffer = []
      if(json_f["objects"][i]["label"]) not in id_dict_labels: continue
      
      label = id_dict_labels[json_f["objects"][i]["label"]]

      xmin = json_f["objects"][i]["bbox"]["xmin"]
      xmax = json_f["objects"][i]["bbox"]["xmax"] 
      ymin = json_f["objects"][i]["bbox"]["ymin"] 
      ymax = json_f["objects"][i]["bbox"]["ymax"] 

      xc = (xmin+xmax)/2
      yc = (ymin+ymax)/2

      w = xmax-xmin
      h = ymax-ymin

      buffer.append(str(label))
      buffer.append(str(xc/width))
      buffer.append(str(yc/height))
      buffer.append(str(w/width))
      buffer.append(str(h/height))
      #print(buffer)

      #write on file
      f.write(buffer[0]+' '+buffer[1]+' '+buffer[2]+' '+buffer[3]+' '+buffer[4]+'\n')
    f.close()
  print("number of total files", k)

In [ ]:
#img_path = ""
label_path = ""

j = 0

id_labels = {}

label_path = "/mnt/c/Users/aless/Desktop/ex_yolo/Mapillary/MAPILLARY_full_train_0/labels_dataset/"
convert_value_2_txt(label_path, id_labels)  